# **Instructions for downloading and processing Vence data using python API.**

**API Code written by Andrew Antaya University of Arizona**

**Data processing code written by Jameson Brennan South Dakota State University**

This is a google colab tutorial for downloading Vence API data. Users must have Vence database setup to utilize the code. In addition, a google drive folder must be setup correctly to read in the VF_In_Service.py file containing the API code and the correct output folders setup to save the data. Please view the readme file to see how to setup.

1. The cells outlined in gray are the ones which contain chunks of code to run. These will process the data, and give us our outputs. In order to run the line of code, move your cursor to the grey box and run ht cell by clicking on the 'play' triangle symbol or by clicking in hte cell and pressing ctrl + enter. For example, click in the grey cell below and run the cell and you should see the output of 10 + 10

In [1]:
10 + 10

20

2. Now that we have an idea on how to run chunks of code, we can start to process our vence data. First we want enter our customer name, database user, password, and start and end time. You will need to reach out to Vence to get access to this information as it is different from the login used for the Vence software/  After each line, type in the information, changing the examples after the = sign. Make sure to keep the start and end time in the same format as: YYYY-mm-dd hh:mm:ss.ms or else you will encounter an error below.

In [2]:
customer="test_test"
dbuser="test_test_ro"
dbpassword= "123456789abcd"
start_time= "2022-06-12 12:00:00.000"
end_time="2022-06-15 12:00:00.000"

Next we want to import the python modules we will use to run our analysis.  If you are familiar to Program R, modules in python are similar to libraries in R. Each module contains a set of functions which can be used to process data. One of the main benefits to running the code in google colab is that doesn't require you download any modules to you local hard drive and instead runs everything in the cloud. When you run the code you will need to connect the colab notebook to your google account and allow permission to access your google drive. This will let you download the vence data to your personal google drive account. 

Google Collab saves output onto **your** Google Drive. We need to allow Google Collab permission to access your Google Drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

Next, we will create a folder in your Google Drive called "Vence_API". This is where we will save data retrieved from the API. 

In [ ]:
import sys
import os

path = '/content/drive/My Drive/Vence_API'

if not os.path.isdir('/content/drive/My Drive/Vence_API'):
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

sys.path.append('/content/drive/My Drive/Vence_API')

Next, we will import Python code to run the API. This code chunk reads in another Python file from GitHub and writes it to your Google Drive. This Python file contains functions to run the API and process virtual fence data.

In [7]:
import requests

url = 'https://raw.githubusercontent.com/amantaya/vf-workshop/main/code/api-modified-for-workshop.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('api.py', 'w') as f:
    f.write(r.text)

# now we can import
import api as vf_api

Next, we need to install the required packages to run the API and analysis.

In [8]:
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas
!pip install timezonefinder
!pip install utm
import geopandas
# load required libraries
import os
import sys
import re
import warnings
import requests
from requests.structures import CaseInsensitiveDict
import datetime
import pandas as pd
import json
import numpy as np
import utm
from shapely.geometry import Point
import timezonefinder
import pytz
  #import utm
from shapely.geometry import Point
from geopandas import GeoDataFrame

'apt' is not recognized as an internal or external command,
operable program or batch file.


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached setuptools-67.0.0-py3-none-any.whl (1.1 MB)
  Using cached cffi-1.15.1-cp39-cp39-win_amd64.whl (179 kB)
  Created wheel for timezonefinder: filename=timezonefinder-6.1.9-cp39-cp39-win_amd64.whl size=46067467 sha256=cb7c528c00b7a7f1a18c6a31434bd2792b71787e28e4faa0bfa86677635bae9c
  Stored in directory: c:\users\andre\appdata\local\pip\cache\wheels\34\10\69\7a540f8d5e6701de1096b59790a85c53cc48676053e2d93470
Successfully built timezonefinder
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.2.0
    Uninstalling setuptools-61.2.0:
      Successfully uninstalled setuptools-61.2.0
  Attempting uninstall: cffi
    Found existing installatio

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\andre\\AppData\\Local\\Temp\\pip-uninstall-1ibajlp9\\_cffi_backend.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6092 sha256=fe88b3b10c40b0b364ea053b8de72b468d7f77b0160eee206e4d04d2640ae0a0
  Stored in directory: c:\users\andre\appdata\local\pip\cache\wheels\75\f0\dd\39dfc9436520412399934d4a8566fe5c0cb8cc2568a1196057
Successfully built utm


ModuleNotFoundError: No module named 'timezonefinder'

4. The next line of code will access the API and start downloading the Vence data from the account and dates that you specified below. It is important that the customer, dbuser, and dbpassword used above is entered correctly or you will generate an error. Also important to to ensure the dates you entered are from a date when you were generating Vence data or else the API will return an empty dataframe and cause an error. Depending on how long of a time period you specified earlier and how many active collars, it may take a while to finish downloading all the data. In addition, sometimes error messages may occur if the server times out. In these cases retry the cell again, or try specifying a shorter time frame for data downloads.

In [ ]:
vf_api.Vence_Api(customer,dbuser,dbpassword,start_time,end_time)

5. The next cell will process the parsed Vence messages into individual collar files. In addition it will export each file into a csv and shapefile and save them in the folders specified earlier in google drive. Processing steps added also include calculating duration between successive fixes, converting the date time stamp to local time based on latitude and longitude, calulating distance between successive fixes, calculating rate of travel in m/min, and flagging data that may indicate a collar has fallen off.

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/amantaya/vf-workshop/main/code/processing-vf-message-data.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('vfprocessing.py', 'w') as f:
    f.write(r.text)

# now we can import
import vfprocessing as vf

In [ ]:
vf.process_vf_data(df)

If you check your google drive Vence_API folder the 'csv_files' and 'shp_files' folders you created should now be populated with the individual collar files.